# 3. Data Cleaning Address (TPE)
資料前處理-地址   
3_1. 文字清理   
3_2. 統一地址格式   
3_3. 找出錯誤資訊    
3-4. 無法判讀的資料 (錯字)     
3-5. 合併與統整   
3_6. 合併gis資料

In [1]:
#載入套件
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import missingno as msno

import nums_from_string as nfs
import unicodedata
#from datetime import datetime

#匯入自己寫的程式
import address_cleaning as ac
import address_data_correction as adc
import address_surrounding as asu

In [2]:
df = pd.read_csv('2_data_selection.csv', low_memory=False)
#df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151952 entries, 0 to 151951
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Identifier            151952 non-null  object 
 1   Address               151952 non-null  object 
 2   District              151952 non-null  object 
 3   Type                  151952 non-null  object 
 4   UrbanZoning           151130 non-null  object 
 5   Purpose               151952 non-null  object 
 6   Material              151928 non-null  object 
 7   NumberOfTransactions  151952 non-null  object 
 8   Storeys               151637 non-null  object 
 9   Floor                 151811 non-null  object 
 10  CompletionDate        140299 non-null  float64
 11  TransactionDate       151952 non-null  int64  
 12  Bedrooms              151952 non-null  int64  
 13  LivingRooms           151952 non-null  int64  
 14  Bathrooms             151952 non-null  int64  
 15  

# 3-1. 文字清理 
清理原始資料(df)   
step1. 全形轉半形   
step2. 補上OO市OO區   
step3. 地址到"樓"

In [3]:
df = ac.address_text(df, '台北市', 1)

缺失值數量: 0
Cleaning address text...
Finish!
Missing city: 17847,  Split_樓: 138651,  Split_及/;: 54
缺失值數量: 0


In [4]:
df.to_csv('address_text.csv', encoding='utf-8-sig', index=False)

# 3-2. 統一地址格式
清理新表(data)    
step1. 建立新表data，將地址切詞後存入   
step2. 一個個欄位清理內文，將數字統一成中文/阿拉伯數字   
step3. "號": 保留第一項 (3之4號->3號)，"樓": 保留最後一項 (1至4樓->4樓)   

In [5]:
df = pd.read_csv('address_text.csv', low_memory=False)

In [6]:
data = ac.address_creat_dataframe(df)

Address Creat a dataframe and clean it...
step1. Parse address data...
finished step1
step2. Format address numbers (Chinese/Arabic)...
finished step2
step3. Cleaning the text of each segments...
finished step3
Finish!


In [7]:
data.to_csv('address_creat_dataframe.csv', encoding='utf-8-sig', index=False)

# 3-3. 找出錯誤資訊
地址內的'OO區'與欄位'District'不同，需查證 (新表data)   
step1. 根據郵局網站資料，建立"區"與"路名"的字典   
step2. 比對郵局資料，修正District 

In [8]:
#df = pd.read_csv('address_creat_dataframe.csv', low_memory=False)

In [9]:
# Find Incorrect Information
# (a) The district in the 'Address' column does not match the 'District' column (requiring verification)
data = adc.address_district_error(data)
data.to_csv('address_district_error.csv', encoding='utf-8-sig', index=False)

Find the rows where "District" and "addr_district" do not match...
step1. Making dict_road...
finish step1
Error: '公館路'
北投區 士林區 公館路
Error: '公館路'
北投區 士林區 公館路
Finish!
district_error count: 267
post: 0, orig: 247, addr: 2, unknown: 16


# 3-4. 無法判讀的資料 (錯字)
處理無法判讀的資料 (新表data)   
step1. 處理常見錯字: "公舘路" and "公館街"    
step2. 根據郵局網站資料，建立"路名"與"區"的字典   
step3. 有亂碼的路名，對應字典修正      
step4. 檢查不在字典中的路名 (通常為錯字)   
step5. 錯字與路名更正 ('軍功路'更名為'和平東路四段')

In [10]:
data = pd.read_csv('address_district_error.csv', low_memory=False)

In [11]:
# (b) Misspelled or missing characters, which make the text unreadable
data['road'].replace('ㄧ', '一', inplace=True)
data = adc.address_misspelled(data)

Correcting the misspelled road names...
step1. Formatng "公舘路" and "公館街"...
finish step1!
step2. Make a dictionary of districts...
finish step2!
step3. Find the correct road names...
finish step3!
Finish!


In [12]:
data.to_csv('address_misspelle.csv', encoding='utf-8-sig', index=False)

In [13]:
data = pd.read_csv('address_misspelle.csv', low_memory=False)
rows_to_check, rows_to_check_list = adc.find_not_post(data)

Check the roads not in postal data...
unique roads to be checked: 60
total number of rows to be checked: 82
Finish. Save the datas as data_check.csv


In [14]:
set(rows_to_check_list)

{'中山北路',
 '內湖路',
 '內胡路二段',
 '六段',
 '南京南路五段',
 '南京東路',
 '南路三段',
 '塔攸路',
 '安康街',
 '幸亥路三段',
 '康欒街',
 '廷吉街',
 '忠孝東五段',
 '忠孝東六段',
 '忠誠路ㄧ段',
 '承德路',
 '政大二路',
 '文德街',
 '文林路一段',
 '文湖路',
 '文湖路二段',
 '新北路二段',
 '新生北路',
 '新興街',
 '木新路',
 '木柵路ㄧ段',
 '民全東路六段',
 '民權東路',
 '民權東路陸段',
 '民生東路',
 '永吉路三段',
 '汀洲路ㄧ段',
 '汀洲路一段',
 '汀洲路二段',
 '溪州街',
 '直興段',
 '研究院二段',
 '研究院路',
 '社中路',
 '福洲街',
 '福港路',
 '立農街',
 '紹安街',
 '羅斯福二段',
 '羅斯福路',
 '興南路',
 '螺斯福路三段',
 '複興南路二段',
 '褔林路',
 '西園路',
 '西安路二段',
 '豐年路',
 '貴陽路二段',
 '路二段',
 '路五段',
 '還山路二段',
 '重慶北路',
 '錦洲街',
 '門街',
 '麗山路'}

In [15]:
data = adc.misspelled_roads(data)
rows_to_check, rows_to_check_list = adc.find_not_post(data)
set(rows_to_check_list)

Correcting the misspelled roads...
finish step5
Check the roads not in postal data...
unique roads to be checked: 52
total number of rows to be checked: 66
Finish. Save the datas as data_check.csv


{'中山北路',
 '內湖路',
 '內胡路二段',
 '六段',
 '南京南路五段',
 '南京東路',
 '南路三段',
 '塔攸路',
 '安康街',
 '幸亥路三段',
 '康欒街',
 '廷吉街',
 '忠孝東五段',
 '忠孝東六段',
 '承德路',
 '政大二路',
 '文德街',
 '文林路一段',
 '文湖路',
 '文湖路二段',
 '新北路二段',
 '新生北路',
 '新興街',
 '木新路',
 '民全東路六段',
 '民權東路',
 '民權東路陸段',
 '民生東路',
 '永吉路三段',
 '直興段',
 '研究院二段',
 '研究院路',
 '社中路',
 '福港路',
 '立農街',
 '紹安街',
 '羅斯福二段',
 '羅斯福路',
 '興南路',
 '螺斯福路三段',
 '複興南路二段',
 '褔林路',
 '西園路',
 '西安路二段',
 '豐年路',
 '貴陽路二段',
 '路二段',
 '路五段',
 '還山路二段',
 '重慶北路',
 '門街',
 '麗山路'}

In [16]:
data.to_csv('address_misspelle.csv', encoding='utf-8-sig', index=False)

# 3-5. 合併與統整
將新表data的資料，合併到原資料df裡

In [4]:
df = pd.read_csv('address_text.csv', low_memory=False)
data = pd.read_csv('address_misspelle.csv', low_memory=False)

In [5]:
df = adc.address_format_to_all(df, data)

Address format...
Drop addresses that cannot be recognized...
Drop: 20
Finish!


In [6]:
# Drop the 'District' column
df.drop(['District'], axis=1, inplace=True)
df.rename(columns={'addr_district': 'District'}, inplace=True)

In [7]:
df.to_csv('address_format_to_all.csv', encoding='utf-8-sig', index=False)

# 3-6. "地域環境"資料

In [2]:
#df = pd.read_csv('address_format_to_all.csv', low_memory=False)

In [3]:
df = asu.coordinates(df)

Convert Addresses to Latitude and Longitude...
Before--df.shape: (151932, 30)
Duplicate IDs: 0
Finish!
After--df.shape: (151932, 32)


In [4]:
df = asu.surroundings_500m(df)

Merge with Surrounding DataFrames (500m Radius)...
Before--df.shape: (151932, 32)
Num_NIMBY (number of duplicate IDs): 0
Num_Hospital (number of duplicate IDs): 0
Num_Metro (number of duplicate IDs): 0
Num_Railway (number of duplicate IDs): 0
Num_Primary_School (number of duplicate IDs): 0
Num_Sec_School (number of duplicate IDs): 0
Finish!
After--df.shape: (151932, 38)


In [5]:
df = asu.surroundings_nearest(df)
#df.to_csv('surroundings_nearest.csv', encoding='utf-8-sig', index=False)

Merge with Surrounding DataFrames (Nearest)...
Before--df.shape: (151932, 38)
NIMBY_Station (number of duplicate IDs): 0
Hospital (number of duplicate IDs): 0
Metro_Station (number of duplicate IDs): 0
Railway (number of duplicate IDs): 0
Primary_School (number of duplicate IDs): 0
Secondary_School (number of duplicate IDs): 0
Finish!
After--df.shape: (151932, 50)


In [8]:
#查看各欄位缺失植數量
missing_values = df.isna().sum()
missing_value_true = missing_values[missing_values > 1]
print(missing_value_true)

UrbanZoning          822
Material              24
Storeys              315
Floor                141
CompletionDate     11651
Elevator          106151
UnitPrice             19
addr_floor         13331
dtype: int64


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151932 entries, 0 to 151931
Data columns (total 50 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Identifier            151932 non-null  object 
 1   Address               151932 non-null  object 
 2   Type                  151932 non-null  object 
 3   UrbanZoning           151110 non-null  object 
 4   Purpose               151932 non-null  object 
 5   Material              151908 non-null  object 
 6   NumberOfTransactions  151932 non-null  object 
 7   Storeys               151617 non-null  object 
 8   Floor                 151791 non-null  object 
 9   CompletionDate        140281 non-null  float64
 10  TransactionDate       151932 non-null  int64  
 11  Bedrooms              151932 non-null  int64  
 12  LivingRooms           151932 non-null  int64  
 13  Bathrooms             151932 non-null  int64  
 14  Partitions            151932 non-null  object 
 15  

In [10]:
df.to_csv('3_data_cleaning_address.csv', encoding='utf-8-sig', index=False)